%%capture
!pip install import-ipynb
!pip install textblob
!pip install spacy
!pip install nltk
!pip install -U sklearn
!pip install googletrans==3.1.0a0

In [1]:
import pandas as pd
import import_ipynb
import numpy as np
import spacy
import os
import re
import nltk

from Preprocessing import *
from leia import SentimentIntensityAnalyzer
from textblob import TextBlob
from textblob.classifiers import NaiveBayesClassifier
from sklearn.metrics import accuracy_score
from googletrans import Translator

translator = Translator(service_urls=['translate.googleapis.com'])
nltk.download('punkt')
nlp = spacy.load('pt_core_news_sm')

importing Jupyter notebook from Preprocessing.ipynb


[nltk_data] Downloading package punkt to /home/greca/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## Models

### LeIA

In [2]:
analyzer = SentimentIntensityAnalyzer()
def leia(text):
    text = str(text)
    result = analyzer.polarity_scores(text)
    
    #analisa a frase utilizando o compound
    if result['compound'] >= 0.05:
        return 'positivo'
    elif result['compound'] <= -0.05:
        return 'negativo'
    else:
        return 'neutro'

### TextBlob + ReLi

In [3]:
def textblob(sentence):
    sentence = str(sentence)
    blob = TextBlob(sentence)
    result = 0
    
    #traduz o texto (em português) para inglês
    translation = translator.translate(sentence, src='pt', dest='en')
    translation = translation.text
    translation = TextBlob(translation)
    result = translation.sentiment.polarity
        
    if result > 0:
        return 'positivo'
    elif result < 0:
        return 'negativo'
    else:
        return 'neutro'

### OpLexicon

In [4]:
with open('lexico_v3.0.txt', 'r') as f:
    lines = f.readlines()

lines = [str(x.strip()) for x in lines]
pol_dict = {}

for line in lines:
    word, _, pol, _ = line.split(',')
    
    if word not in pol_dict.keys():
        pol_dict[word] = pol

In [5]:
def oplexion(text):
    text = str(text)
    doc = nlp(text)
    pol = 0
    
    for token in doc:
        if token.text in pol_dict.keys():
            if token.pos_ == 'VERB':
                if token.lemma_ in pol_dict.keys():
                    pol += int(pol_dict[str(token.lemma_)])
                else:
                    pol += int(pol_dict[str(token.text)])
            else:
                pol += int(pol_dict[str(token.text)])
        else:
            pol += 0
        
    if pol > 0:
        return 'positivo'
    elif pol < 0:
        return 'negativo'
    else:
        return 'neutro'

### SentiLex

In [6]:
with open('SentiLex-lem-PT01.txt', 'r') as f:
    lines = f.readlines()

lines = [str(x.strip()) for x in lines]
pol_dict = {}

for line in lines:
    word, infos = line.split('.')
    pol = infos.split(';')
    pol = pol[3]
    pol = pol[4:]
    
    if word not in pol_dict.keys():
        pol_dict[word] = pol

In [7]:
def sentilex(text):
    text = str(text)
    doc = nlp(text)
    pol = 0
    
    for token in doc:
        try:
            if token.pos_ == 'VERB':
                pol += int(pol_dict[str(token.lemma_)])
            else:
                pol += int(pol_dict[str(token.text)])
        except KeyError:
            pol += 0
        
    if pol > 0:
        return 'positivo'
    elif pol < 0:
        return 'negativo'
    else:
        return 'neutro'

In [8]:
data = pd.read_csv('data.csv')
data.head()

,id,tweet_text,tweet_date,sentiment,query_used
0,1031761728445530112,@Tixaa23 14 para eu ir :),Tue Aug 21 04:35:39 +0000 2018,Positivo,:)
1,1031761040462278656,@drexalvarez O meu like eu já dei na época :),Tue Aug 21 04:32:55 +0000 2018,Positivo,:)
2,1031760962372689920,Eu só queria conseguir comer alguma coisa pra ...,Tue Aug 21 04:32:37 +0000 2018,Positivo,:)
3,1031760948250456066,:D que lindo dia !,Tue Aug 21 04:32:33 +0000 2018,Positivo,:)
4,1031760895985246208,"@Primo_Resmungao Pq da pr jeito!!é uma ""oferta...",Tue Aug 21 04:32:21 +0000 2018,Positivo,:)


In [9]:
data.shape

(785814, 5)

In [10]:
data = data.drop(columns=['id', 'tweet_date', 'query_used'])
data = data.rename({'tweet_text': 'text', 'sentiment': 'target'}, axis=1)
data['target'] = data['target'].apply(lowercase)
data.head()

,text,target
0,@Tixaa23 14 para eu ir :),positivo
1,@drexalvarez O meu like eu já dei na época :),positivo
2,Eu só queria conseguir comer alguma coisa pra ...,positivo
3,:D que lindo dia !,positivo
4,"@Primo_Resmungao Pq da pr jeito!!é uma ""oferta...",positivo


In [11]:
%%time
data = data.sample(500)
data = data.reset_index()
data = data.drop(columns=['index'])
data['cleaned_text'] = preprocessing(data['text'])
data.drop_duplicates(inplace=True)
data.dropna(inplace=True)
data.head(10)

CPU times: user 89.4 ms, sys: 3.72 ms, total: 93.1 ms
Wall time: 89.9 ms


,text,target,cleaned_text
0,to morrendo de tanta vontade de assistir para ...,negativo,to morrendo tanta vontade assistir todos garot...
1,"@sdRobertoJr é ossada né, fazer o que :(",negativo,ossada ne fazer cara triste
2,@CELLZAS difícil nao escutar quando falam isso...,negativo,dificil nao escutar falam praticamente todo di...
3,pessoal eu tô vendendo esses livros aqui pra q...,positivo,pessoal to vendendo livros aqui pra apaixonado...
4,@_mandsborba vai me magoar nene :(,negativo,vai magoar nene cara triste
5,sdd n ta cabendo em mim :(,negativo,sdd n ta cabendo mim cara triste
6,@danibarrichello Obrigado! :))))),positivo,obrigado cara feliz
7,Até o meu cão dorme no meu quarto e eu não :),positivo,cao dorme quarto cara feliz
8,@howcoldplay não não não eu amo mais :),positivo,amo cara feliz
9,"@xxxcheara Jiyong é um neném, ele precisa ser ...",negativo,jiyong nenem precisa ser protegido atencao ca...


In [12]:
data.shape

(500, 3)

In [13]:
%%time
data['leia'] = data['cleaned_text'].apply(leia)

CPU times: user 125 ms, sys: 7.17 ms, total: 132 ms
Wall time: 130 ms


In [14]:
%%time
#se usar o texto limpo a acurácia fica muito baixa
#isso acontece por causa da api não conseguir direito para inglês
data['textblob'] = data['text'].apply(textblob)

CPU times: user 5.89 s, sys: 286 ms, total: 6.17 s
Wall time: 4min 2s


In [15]:
%%time
data['oplexion'] = data['cleaned_text'].apply(oplexion)

CPU times: user 4.6 s, sys: 0 ns, total: 4.6 s
Wall time: 4.62 s


In [16]:
%%time
data['sentilex'] = data['cleaned_text'].apply(sentilex)

CPU times: user 4.36 s, sys: 0 ns, total: 4.36 s
Wall time: 4.37 s


In [17]:
print("LeIA: " + str(accuracy_score(data['target'], data['leia'])))
print("TextBlob: " + str(accuracy_score(data['target'], data['textblob'])))
print("OpLexion: " + str(accuracy_score(data['target'], data['oplexion'])))
print("SentiLex: " + str(accuracy_score(data['target'], data['sentilex'])))

LeIA: 0.916
TextBlob: 0.996
OpLexion: 0.898
SentiLex: 0.914


In [18]:
data.head(10)

,text,target,cleaned_text,leia,textblob,oplexion,sentilex
0,to morrendo de tanta vontade de assistir para ...,negativo,to morrendo tanta vontade assistir todos garot...,negativo,negativo,positivo,positivo
1,"@sdRobertoJr é ossada né, fazer o que :(",negativo,ossada ne fazer cara triste,negativo,negativo,negativo,negativo
2,@CELLZAS difícil nao escutar quando falam isso...,negativo,dificil nao escutar falam praticamente todo di...,negativo,negativo,negativo,negativo
3,pessoal eu tô vendendo esses livros aqui pra q...,positivo,pessoal to vendendo livros aqui pra apaixonado...,positivo,positivo,positivo,positivo
4,@_mandsborba vai me magoar nene :(,negativo,vai magoar nene cara triste,negativo,negativo,negativo,negativo
5,sdd n ta cabendo em mim :(,negativo,sdd n ta cabendo mim cara triste,negativo,negativo,negativo,negativo
6,@danibarrichello Obrigado! :))))),positivo,obrigado cara feliz,positivo,positivo,neutro,positivo
7,Até o meu cão dorme no meu quarto e eu não :),positivo,cao dorme quarto cara feliz,positivo,positivo,positivo,positivo
8,@howcoldplay não não não eu amo mais :),positivo,amo cara feliz,positivo,positivo,positivo,positivo
9,"@xxxcheara Jiyong é um neném, ele precisa ser ...",negativo,jiyong nenem precisa ser protegido atencao ca...,negativo,negativo,negativo,negativo
